In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [3]:
import numpy as np 
import pandas as pd 
import sys 
from scipy.interpolate import interp1d
from scipy import interpolate

sys.path.append('/home/ak/Documents/Research/hsmm/hsmm/')
#sys.path.append('/home/kman/Research/hsmm/')
from hsmm_core.hmm import * 
from hsmm_core.hmm_utils import load_data
from hsmm_core.prediction_engines import *  
import hsmm_core.observation_models 
from test_hmm.test_utils import generate_sample_paths_independent_sets
%matplotlib inline
import matplotlib.pyplot as plt
#plt.style.use('fivethirthyeight')
plt.style.available
import seaborn as sns

ImportError: No module named hsmm_core.hmm

Let us create a fairly distinguishable situation : Two states with skewed transition matrix favouring one state 
and static parameters that are sufficiently apart

In [3]:
n_hidden_states = 2
sigmas = [0.1, 4] 
lambdas = [0.5, 5] 
weights = [0.1, 0.1] 

tpm = np.array([[0.8, 0.2], [0.8, 0.2]])
startprob = np.array([0.8, 0.2]) 
obs_model = hsmm_core.observation_models.ExpUniGauss(n_hidden_states)
obs_model.sigmas_ = sigmas  
obs_model.lambdas_ = lambdas  
obs_model.weight_ = weights 

the_hmm = hmm_engine(obs_model, n_hidden_states) 

the_hmm.pi_ = startprob
the_hmm.tpm_ = tpm

In [4]:
print "sigmas ", obs_model.sigmas_
print "lambdas ", obs_model.lambdas_

print "Transition matrix ", the_hmm.tpm_

sigmas  [0.1, 4]
lambdas  [0.5, 5]
Transition matrix  [[0.8 0.2]
 [0.8 0.2]]


In [2]:
def states_from_fixed_ratios(ratios, total_length):
    states = np.array([], dtype=np.int64) 
    ratios_ids = np.arange(len(ratios))
    rng = np.random.RandomState(345)
    while len(states) < total_length: 
        
        ratio = ratios[rng.choice(ratios_ids)] 
        #print ratio
        states = np.append(states, np.append(np.repeat(0, 100*ratio[0]), np.repeat(1, 100*ratio[1]))) 
        #print len(states)
    return states

# Lets fix a ratio of 20:80 and sample some fixed states 
state_ratios = np.array([[0.2, 0.05], [0.4, 0.1], [0.8, 0.2]]) 

Ts = [1, 5, 10, 20, 50,60, 70, 100,150, 200,300,350, 400,450,500,550, 600,650, 700,750,800,850, 900,1000,1100, 1200,1300,1400, 1500]
M = 1000

all_errors = np.zeros((len(Ts), M))

NameError: name 'np' is not defined

In [1]:
for i_T, T in enumerate(Ts):
    print i_T, T

NameError: name 'Ts' is not defined

In [7]:
###create sequences###
for i_T, T in enumerate(Ts):
    rng = np.random.RandomState(12345)
    states = states_from_fixed_ratios(state_ratios, T)
    # uncomment here to generate states from the hidden process distribution directly. 
    # states, _ = the _hmm.sample_states(rng=rng, length=T)
    for m in xrange(M):
        obs_samples = obs_model.sample_data(states, rng=rng)
        
        viterbi_inferred = the_hmm.map_metrics(obs_samples)['viterbi_optimal_state_seq'] 
        all_errors[i_T, m] = np.sum((viterbi_inferred - states)**2)/float(T)  
   # print ("sequence no:",i_T,"length of sequence:",T,"copies of sequence:",M)
   # print'###ERROR###'
   # print all_errors[i_T,m]
   # print '####'

We create state sequences of fixed length 100, 400, 700, 900, 1200, 1500 and for every fixed length generate 
10e4 copies of observation data from the above distribution. We then plot for some length the mean square error of the 
viterbi inference algorithm. Namely, if $states$ our the original states and $O_{1:T}^{(m)}$ is the m-th batch, the Viterbi inference error for this batch is $$\epsilon^{(m)}(T) := \frac{1}{T}\sum_{t=1}^T (Vstates^{(m)}_i - states_i)^2$$ where $Vstates$ are the states inferred by the Viterbi algo for the observation batch $O^{(m)}_{1:T}$. The total MC error for a given $T$ is then 
$$\epsilon(T) := \frac{1}{M} \sum_{m=1}^M (\epsilon^{(m)}(T) - \bar{\epsilon}(T))^2, \quad \bar{\epsilon}(T):=  \frac{1}{M}\sum_{m=1}^M \epsilon^{(m)}(T)$$

In [ ]:
error_means_wrt_m = np.mean(all_errors,axis=1) 
final_errors = np.mean((all_errors - error_means_wrt_m[:, np.newaxis])**2, axis=1)

In [ ]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12

As predicted, if I fix the length of the states (time horizon T) and plot the first error with respect to $m$ (the number of observation copies), the error will decrease to a certan lower threshold.  

In [1]:
total_error_fixedT = np.cumsum(all_errors, axis=1)
total_error_fixedT /= np.arange(1, M+1)
plt.style.use('seaborn-dark')
sns.set()
#plt.plot(total_error/np.arange(M))
error_T100 = total_error_fixedT[5]
error_T700 = total_error_fixedT[11] 
error_T1500 = total_error_fixedT[-1] 
with plt.style.context('ggplot'):
    # Set an aspect ratio
    width, height = plt.figaspect(0.68)
    fig = plt.figure(figsize=(width,height), dpi=400)
    fig.patch.set_facecolor('white')
    #fig = plt.figure(figsize=(12,8))
    plt.plot(error_T100, color='grey',linestyle='--', label='T100')
    plt.plot(error_T700, color='blue',linestyle=':', label='T700')
    plt.plot(error_T1500, color='green',linestyle='-', label='T1500')

plt.legend(loc='upper right')
plt.xlabel('Sequence Length')
plt.ylabel('Error')

NameError: name 'np' is not defined

In [ ]:
#sns.tsplot(data=error_T100,err_style="boot_traces", n_boot=500, color="m")


In [71]:
_data_=pd.DataFrame(total_error_fixedT.T)
#_data_.mean(axis=1)
_data_

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.015000,0.015000,0.014000,0.021667,0.030000,0.027778,0.024167,0.024615,0.025000,0.024667
1,0.000000,0.000000,0.000000,0.000000,0.010000,0.025000,0.015000,0.032500,0.019000,0.024167,0.025000,0.026111,0.025833,0.027308,0.027857,0.026667
2,0.000000,0.000000,0.000000,0.000000,0.020000,0.020000,0.020000,0.029167,0.018667,0.026111,0.026667,0.025185,0.028333,0.028462,0.029048,0.026667
3,0.250000,0.050000,0.025000,0.012500,0.020000,0.017500,0.021250,0.030625,0.020000,0.027083,0.026071,0.027222,0.026042,0.024615,0.027679,0.026000
4,0.400000,0.080000,0.040000,0.020000,0.020000,0.022000,0.023000,0.028500,0.020400,0.029333,0.028857,0.026667,0.026500,0.027385,0.027714,0.026133
5,0.500000,0.100000,0.050000,0.025000,0.016667,0.026667,0.025833,0.030000,0.022667,0.030000,0.027619,0.025926,0.027222,0.028205,0.026310,0.026111
6,0.428571,0.085714,0.042857,0.021429,0.017143,0.027143,0.025714,0.031071,0.024857,0.029286,0.025510,0.026984,0.027381,0.027253,0.027347,0.025429
7,0.500000,0.100000,0.050000,0.025000,0.017500,0.028750,0.026875,0.032813,0.024000,0.027292,0.026607,0.027361,0.027396,0.027019,0.026786,0.025417
8,0.444444,0.088889,0.044444,0.022222,0.017778,0.028889,0.028333,0.031667,0.022889,0.029074,0.026984,0.026790,0.027130,0.027009,0.027063,0.025556
9,0.500000,0.100000,0.050000,0.025000,0.020000,0.027000,0.030000,0.030750,0.022600,0.028833,0.028286,0.026778,0.027250,0.027769,0.027143,0.025267


However, plotting the total error with respect to sequence length, once averaged out over $10k$ paths, shows the desirable decrease. 


In [2]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111) 
fig.subplots_adjust(top=0.85)
ax.set_ylabel('MC - error', fontsize=14)
ax.set_xlabel('T', fontsize=14)
ax.text(1000, .00025, 'Using 1e4 batches', fontsize=14) 
f2 = interp1d(Ts, final_errors, kind='linear')
f3=interpolate.splrep(Ts, final_errors, s=2)
#ynew = interpolate.splev(xnew, f3, der=1)
ax.plot(Ts, final_errors,'--',color='r')

#xnew = np.linspace(1, 1500, num=11, endpoint=True)
#ax.plot(xnew, f2(xnew))
#ax.plot(xnew, ynew)
#import matplotlib.pyplot as plt
plt.plot(x, y, 'o', xnew, f(xnew), '-', xnew, f2(xnew), '--')
ax.axis([0, 1500, 0., 0.0003] ) ;

NameError: name 'plt' is not defined

In [64]:
data= f3(T)
#sns.tsplot(data=data, err_style="ci_bars", color="g", interpolate=Falsesns.tsplot(data=data, err_style="ci_bars", color="g"))

NameError: name 'Falsesns' is not defined

print final_errors.shape
Ts =np.asarray(Ts)
Ts.shape
# define pts from the question

tck, u = splprep(pts.T, u=None, s=0.0, per=1) 
u_new = np.linspace(u.min(), u.max(), 1000)
x_new, y_new = splev(u_new, tck, der=0)

plt.plot(pts[:,0], pts[:,1], 'ro')
plt.plot(x_new, y_new, 'b--')
plt.show()



In [ ]:
from matplotlib.lines import Line2D
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111) 
fig.subplots_adjust(top=0.85)
ax.set_ylabel('MC - error', fontsize=14)
ax.set_xlabel('T', fontsize=14)
ax.text(1000, .00025, 'Using 1e4 batches', fontsize=14) 
xnew=np.arange(np.min(Ts), np.max(Ts), 20)
x0= Ts
y0= final_errors
ax.axis([0, 1500, 0., 0.0003] ) ;
ax.plot(Ts, final_errors)
linestyles = ['_', '-', '--', ':']
markers = []
for m in Line2D.markers:
    try:
        if len(m) == 1 and m != ' ':
            markers.append(m)
    except TypeError:
        pass
options= ('linear','slinear','zero')

styles = markers + [
    r'$\lambda$',
    r'$\bowtie$',
    r'$\circlearrowleft$',
    r'$\clubsuit$',
    r'$\checkmark$']

colors = ('b', 'g', 'r', 'c', 'm', 'y', 'k')
for o in options:
    f = interp1d(x0, y0, kind=o)
    plt.plot(xnew, f(xnew), label=o)
    
      # plot of interpolated data
plt.legend()
plt.show()
# Array with points in between those of the data set for interpolation.
# Available options for interp1d

In [5]:
f_eng= hmm_features(hmm=the_hmm)
df = pd.DataFrame(obs_samples, columns=['Duration', 'ReturnTradedPrice'])
features = f_eng.generate_features(df)

NameError: name 'obs_samples' is not defined




for m in range(M):
    fresh_observations = the_hmm.ob_model_.sample(...)
    fresh_features = f_eng.generate_features(fresh_observations)
    the_determinant += Determinant(fresh_features['ksi'])

determinant_mc = the_determinant/float(M) 

In [ ]:
def func_determinant(features, i):
    for i in range(0,3):
        _matrix=np.dot(features[i].T,features[i])
        _determinant = np.linalg.det(_matrix)
    return _determinant
    
def symmetric_matrix(features, i):
    for i in range(0,3):
        _sym_mat[i]= np.dot(features[i].T,features[i])

In [6]:
T_i=50
states = states_from_fixed_ratios(state_ratios, T_i)

In [7]:
def squareform_diagfill(arr1D): 
    n = int(np.sqrt(arr1D.size*2)) 
    if (n*(n+1))//2!=arr1D.size: 
        print "Size of 1D array not suitable for creating a symmetric 2D array!" 
        return None 
    else: 
        R,C = np.triu_indices(n) 
        out = np.zeros((n,n),dtype=arr1D.dtype) 
        out[R,C] = arr1D 
        out[C,R] = arr1D 
        return out

In [16]:
state_ratios = np.array([[0.2, 0.05], [0.4, 0.1], [0.8, 0.2]]) 

Ts = [10, 40, 70, 90, 120, 500]
M = 100

#all_determinants = np.zeros((len(Ts), M))
f_eng= hmm_features(hmm=the_hmm)
_inf_matrix={}
_ksi ={}

for i_T, T in enumerate(Ts):
    rng = np.random.RandomState(12345) #inside to make the sequences nested
    states = states_from_fixed_ratios(state_ratios, T)
    for m in xrange(M):
        _obs_samples = obs_model.sample_data(states, rng=rng)
        _df = pd.DataFrame(_obs_samples, columns=['Duration', 'ReturnTradedPrice'])
        _features = f_eng.generate_features(_df)
        _inf_matrix[i_T, m]=_features[1] #information matrix
        _ksi[i_T, m] =_features[3] #ksi

In [23]:
inf_matrix_keys= _inf_matrix.keys()
_trace ={}
_spectral_gap ={}
_determinant ={}

for key in inf_matrix_keys:
    #for i in range(1,len())
    _inf_matrix_square =squareform_diagfill(np.asarray(_inf_matrix[key][1:2]))
    _s= np.linalg.svd(_inf_matrix, full_matrices=False, compute_uv=False)
    _spectral_gap[key]=_s.max() -_s.min()
    _trace[key] =np.sum(_s)
    _determinant[key] =np.prod(_s)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [21]:
#squareform_diagfill(np.asarray(all_data[5, 34][1:2]))

ksi_keys =_ksi.keys()
for key in ksi_keys:
    i=1
    #for i in range(1,len())
    _ksi_square = squareform_diagfill(np.asarray(_ksi[key][i:i+1]))
    _s_ksi= np.linalg.svd(_inf_matrix, full_matrices=False, compute_uv=False)
    _spectral_gap_ksi[key]=_s.max() -_s.min()
    _trace_ksi[key] =np.sum(_s)
    _determinant_ksi[key] =np.prod(_s)

In [ ]:
square_test =squareform_diagfill(test_)

In [ ]:
_trace

In [ ]:
_s= np.linalg.svd(square_test, full_matrices=False, compute_uv=False)

In [ ]:
for i in range(1,1400):
    try:
        matrix= np.asarray(features[1][i:i+1])
        square_test =squareform_diagfill(matrix)
        _s= np.linalg.svd(square_test, full_matrices=False, compute_uv=False)
        matrix_d[(j,i)] = _s 
    except:
        print i

In [ ]:
sp_gap =[]
_trace =[]
_determinant =[]
for i in range(1,1400):
    try:
        _matrix= np.asarray(features[1][i:i+1])
        square_test =squareform_diagfill(_matrix)
        _s= np.linalg.svd(square_test, full_matrices=False, compute_uv=False)
        spectral_gap = _s.max()- _s.min()
        sp_gap.append(spectral_gap)
        mat_trace= np.sum(_s)
        _trace.append(mat_trace)
    except:
        print i